In [2]:
!pip install kagglehub

   ---------------------------------------- 0.0/63.0 kB ? eta -:--:--
   ------------------------- -------------- 41.0/63.0 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 63.0/63.0 kB 1.1 MB/s eta 0:00:00


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("patrickb1912/ipl-complete-dataset-20082020")

print("Path to dataset files:", path)

100%|██████████| 1.82M/1.82M [00:01<00:00, 1.80MB/s]

Extracting files...


Path to dataset files: C:\Users\DELL\.cache\kagglehub\datasets\patrickb1912\ipl-complete-dataset-20082020\versions\3


In [4]:
import pandas as pd

In [5]:
bowls = pd.read_csv(f"{path}/deliveries.csv")
matches = pd.read_csv(f"{path}/matches.csv")

In [8]:
bowls.head()
matches.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [9]:
pip install pandas scikit-learn xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB 217.9 kB/s eta 0:11:29
   ---------------------------------------- 0.0/150.0 MB 326.8 kB/s eta 0:07:39
   ---------------------------------------- 0.1/150.0 MB 525.1 kB/s eta 0:04:46
   ---------------------------------------- 0.1/150.0 MB 657.1 kB/s eta 0:03:49
   ---------------------------------------- 0.2/150.0 MB 901.1 kB/s eta 0:02:47
   ---------------------------------------- 0.3/150.0 MB 930.9 kB/s eta 0:02:41
   ---------------------------------------- 0.3/150.0 MB 930.9 kB/s eta 0:02:41
   ---------------------------------------- 0.3/150.0 MB 840.2 kB/s eta 0:02:59
   ---------------------------------------- 0.4/150.0 MB 873.6 kB/s eta 0:02:52
   ---------------------------------------- 0.4/150.0 MB 895.2 kB/s eta 0:02:48
   ---------------------------------------- 0.4/150.0 MB 8

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load dataset
df = matches

# Drop only the columns that exist in the DataFrame
columns_to_drop = ['id', 'player_of_match', 'method', 'umpire1', 'umpire2', 'super_over']

# Use `errors='ignore'` to avoid errors if a column is missing
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

print("✅ Columns dropped successfully!")


# Convert 'date' column to numerical (year, month, day)
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df.drop(columns=['date'], inplace=True)  # Drop original date column

# Convert numerical columns to float
df['target_runs'] = pd.to_numeric(df['target_runs'], errors='coerce').fillna(0)
df['target_overs'] = pd.to_numeric(df['target_overs'], errors='coerce').fillna(0)



# Encode categorical columns
label_encoders = {}
categorical_cols = ['season', 'city', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'match_type', 'result', 'winner']

for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col].astype(str))  # Convert to string before encoding

# Features and target variable
X = df.drop(columns=['winner'])  # Input features
y = df['winner']  # Target (Winning Team)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.2f}")


✅ Columns dropped successfully!
✅ Model Accuracy: 0.77


In [ ]:
# Encode categorical columns safely
label_encoders = {}
categorical_cols = ['season', 'city', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'match_type', 'result', 'winner']

for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = df[col].astype(str)  # Convert to string
    df[col] = label_encoders[col].fit_transform(df[col])

# Handle new/unknown values for prediction
def encode_for_prediction(input_data):
    """Encode new match data safely, handling unseen values."""
    for col in categorical_cols:
        if col in input_data:
            if input_data[col] in label_encoders[col].classes_:
                input_data[col] = label_encoders[col].transform([input_data[col]])[0]
            else:
                print(f"⚠️ Warning: Unknown value '{input_data[col]}' for column '{col}'")
                input_data[col] = -1  # Assign a default value for unknown data
    return input_data

# Encode categorical columns, ensuring unseen values do not cause errors
def safe_encode(column, value):
    """Encodes a categorical value safely, adding it if unseen."""
    if value in label_encoders[column].classes_:
        return label_encoders[column].transform([value])[0]
    else:
        # Add new category to the existing encoder
        new_classes = list(label_encoders[column].classes_) + [value]
        label_encoders[column].classes_ = new_classes
        return label_encoders[column].transform([value])[0]

def predict_winner(input_data):
    """Encodes new match data safely and predicts the winner."""
    input_df = pd.DataFrame([input_data])

    # Encode categorical columns
    for col in categorical_cols:
        if col in input_df:
            input_df[col] = safe_encode(col, input_df[col].values[0])

    # Convert numerical columns to float
    input_df['target_runs'] = pd.to_numeric(input_df['target_runs'], errors='coerce').fillna(0)
    input_df['target_overs'] = pd.to_numeric(input_df['target_overs'], errors='coerce').fillna(0)

    # Predict winner
    prediction = model.predict(input_df)[0]
    winner = label_encoders['winner'].inverse_transform([prediction])[0]
    
    print(f"🏆 Predicted Winner: {winner}")



In [19]:
test_case_1 = {
    'season': '2024',
    'city': 'Mumbai',
    'venue': 'Wankhede Stadium',
    'team1': 'Mumbai Indians',
    'team2': 'Chennai Super Kings',
    'toss_winner': 'Mumbai Indians',
    'toss_decision': 'bat',
    'match_type': 'League',
    'result': 'runs',  # This field won't be available for actual predictions
    'target_runs': '180',
    'target_overs': '20'
}
predict_winner(test_case_1)

⚠️ Warning: Unknown value '2024' for column 'season'
⚠️ Warning: Unknown value 'Mumbai' for column 'city'
⚠️ Warning: Unknown value 'Wankhede Stadium' for column 'venue'
⚠️ Warning: Unknown value 'Mumbai Indians' for column 'team1'
⚠️ Warning: Unknown value 'Chennai Super Kings' for column 'team2'
⚠️ Warning: Unknown value 'Mumbai Indians' for column 'toss_winner'
⚠️ Warning: Unknown value 'bat' for column 'toss_decision'
⚠️ Warning: Unknown value 'League' for column 'match_type'
⚠️ Warning: Unknown value 'runs' for column 'result'


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:target_runs: object, target_overs: object